In [1]:
library(tidyverse)
library(Seurat)

source('milo_cellbender.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




In [2]:
other_clusters = qs::qread('../00_cellbender/_targets/objects/obj_cb_other_01') %>% 
    set_labels_to_lvl1 %>%
    set_batch_to_lane %>%    
    prep_obj_for_milo_cb_v01 %>%
    `[[` %>%
    pull(labels) %>% 
    unique
other_clusters

[1] "Tanycytes"         "OPC"               "Astrocytes"       
[4] "Ependymal_cells"   "Oligodendrocytes"  "Microglia"        
[7] "Endothelial_cells"

In [3]:
neuron_clusters = qs::qread('../00_cellbender/_targets/objects/obj_cb_neuron_00') %>% 
    set_labels_to_lvl1 %>%
    set_batch_to_lane %>%    
    prep_obj_for_milo_cb_v01 %>%
    `[[` %>%
    pull(labels) %>% 
    unique
neuron_clusters 

[1] "Sst__Pthlh"          "Nfix__Nfib"          "Met__Prlh"          
 [4] "Agrp"                "Sim1"                "Fez1"               
 [7] "Pou6f2__Prkcq__Whrn" "Dlk1"                "Slc17a6__Trhr"      
[10] "Esr1"                "Tmem215__Dach2"      "Sst__Nts"           
[13] "Sst__Unc13c"         "Irx5__Lmx1a"         "Htr3b"              
[16] "Hdc"                 "Lepr89"              "Nfib_1"             
[19] "Ppp1r17__Tafa4"      "Lepr67"              "Arx__Nr5a2"         
[22] "Kiss1__Tac2"         "Met__Pappa2"         "Trh__Lef1"          
[25] "Gpr50__Tac2"         "Tmem215__Greb1"      "Ghrh"               
[28] "Trh__Cxcl12"         "Lepr15"              "Pomc__Ttr"          
[31] "Tbx19__Pirt"         "Pomc__Glipr1"        "Pomc__Anxa2"        
[34] "Npsr1__Epha3"        "Nfix__Nfib__Lhx6"    "Reln__Lef1"         
[37] "unassigned__2__"     "Th__Nfib"            "Ppp1r17"            
[40] "Th__Slc6a3"          "Nfib_3"              "Ppp1r17__Ltbp1"     
[43] "Lpar1_oligo"         "Nfib_2"              "Nfix__Nfib__Hgf"

In [4]:
day_splits = tibble(object = rlang::syms(c("exp_labelled_other", "exp_labelled_neuron")))
days = tibble(day = c("Day5", "Day14", "all"))
day_splits = tidyr::crossing(day_splits, days)
day_splits

object,day
<list>,<chr>
exp_labelled_other,all
exp_labelled_other,Day14
exp_labelled_other,Day5
exp_labelled_neuron,all
exp_labelled_neuron,Day14
exp_labelled_neuron,Day5


In [5]:
other_clusters_tibble = tibble(label = other_clusters) %>% 
tidyr::crossing(day_splits) %>%
filter(str_detect(as.character(object), 'other'))


neuron_clusters_tibble = tibble(label = neuron_clusters) %>% 
tidyr::crossing(day_splits) %>%
filter(str_detect(as.character(object), 'neuron'))

clusters_tibble = rbind(other_clusters_tibble, neuron_clusters_tibble)
clusters_tibble = clusters_tibble %>% 
mutate(cluster = label) %>%
rowwise %>%
mutate(label = case_when(day != "all" ~ paste0(c(day, as.character(label)), collapse='.'),
                          TRUE ~ paste0(label, collapse='.'))) %>%
filter(!str_detect(cluster, 'unassigned'))
clusters_tibble

label,object,day,cluster
<chr>,<list>,<chr>,<chr>
Astrocytes,exp_labelled_other,all,Astrocytes
Day14.Astrocytes,exp_labelled_other,Day14,Astrocytes
Day5.Astrocytes,exp_labelled_other,Day5,Astrocytes
Endothelial_cells,exp_labelled_other,all,Endothelial_cells
Day14.Endothelial_cells,exp_labelled_other,Day14,Endothelial_cells
Day5.Endothelial_cells,exp_labelled_other,Day5,Endothelial_cells
Ependymal_cells,exp_labelled_other,all,Ependymal_cells
Day14.Ependymal_cells,exp_labelled_other,Day14,Ependymal_cells
Day5.Ependymal_cells,exp_labelled_other,Day5,Ependymal_cells


In [6]:
qs::qsave(clusters_tibble, 'clusters_tibble_lvl1.qs')